# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

Provide a company name and their primary website.

In [7]:
# Import necessary libraries
import os
import requests
import json
from typing import List, Dict
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin

load_dotenv()
api_key=os.getenv('DEEPSEEK_API')

if api_key and api_key[:8]=='sk-d07df':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='deepseek-chat'
url = "https://api.deepseek.com"


openai = OpenAI(api_key=api_key, base_url=url)

class Website:
    """
    A class to represent a website and its contents.
    """
    def __init__(self, url: str):
        """
        Initialize the Website object with a given URL.
        
        :param url: The URL of the website to scrape
        """
        self.url = url
        self.title, self.text, self.links = self._scrape_website()

    def _scrape_website(self) -> tuple:
        """
        Scrape the website content, extracting title, text, and links.
        
        :return: A tuple containing the title, text content, and links of the website
        """
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.title.string if soup.title else "No title found"
        
        # Extract text content
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()  # Remove unwanted tags
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        
        # Extract links
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        
        return title, text, links

    def get_contents(self) -> str:
        """
        Get a formatted string of the website contents.
        
        :return: A string containing the website title and text content
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

class LinkAnalyzer:
    """
    A class to analyze and categorize links from a website.
    """
    # System prompt for the OpenAI model to categorize links
    LINK_SYSTEM_PROMPT = """
    You are provided with a list of links found on a webpage. Your task is to first categorize each link into one of the following categories:
    - about page
    - careers page
    - terms of service
    - privacy policy
    - contact page
    - other (please specify).

    Once the links are categorized, please choose which links are most relevant to include in a brochure about the company. 
    The brochure should only include links such as About pages, Careers pages, or Company Overview pages. Exclude any links related to Terms of Service, Privacy Policy, or email addresses.

    Respond in the following JSON format:
    {
        "categorized_links": [
            {"category": "about page", "url": "https://full.url/about"},
            {"category": "careers page", "url": "https://full.url/careers"},
            {"category": "terms of service", "url": "https://full.url/terms"},
            {"category": "privacy policy", "url": "https://full.url/privacy"},
            {"category": "other", "specify": "contact page", "url": "https://full.url/contact"}
        ],
        "brochure_links": [
            {"type": "about page", "url": "https://full.url/about"},
            {"type": "careers page", "url": "https://full.url/careers"}
        ]
    }

    Please find the links below and proceed with the task:

    Links (some may be relative links):
    [INSERT LINK LIST HERE]
    """

    @staticmethod
    def get_links(website: Website) -> Dict:
        """
        Analyze and categorize links from a given website.
        
        :param website: A Website object containing the links to analyze
        :return: A dictionary containing categorized links and brochure-relevant links
        """
        # Prepare the user prompt for the OpenAI model
        user_prompt = f"Here is the list of links on the website of {website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)

        # Make an API call to OpenAI for link analysis
        completion = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": LinkAnalyzer.LINK_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)

class BrochureGenerator:
    """
    A class to generate a company brochure based on website content.
    """
    # System prompt for the OpenAI model to generate the brochure
    SYSTEM_PROMPT = """
    You are an assistant that analyzes the contents of several relevant pages from a company website 
    and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.
    Include details of company culture, customers and careers/jobs if you have the information.
    Structure the brochure to include specific sections as follows:
    About Us
    What we do
    How We Do It
    Where We Do It
    Our People
    Our Culture
    Connect with Us.
    Please provide four versions of the brochure, the first in English, the second in Japanese, the third in Chinese, the fourth in Spanish. The contents of the brochure are to be the same for both languages.
    """

    @staticmethod
    def get_all_details(url: str) -> str:
        """
        Gather all relevant details from a company's website.
        
        :param url: The URL of the company's main page
        :return: A string containing all relevant website content
        """
        result = "Landing page:\n"
        website = Website(url)
        result += website.get_contents()

        # Analyze links and get brochure-relevant ones
        links = LinkAnalyzer.get_links(website)
        brochure_links = links.get('brochure_links', [])
        print("Found Brochure links:", brochure_links)

        # Gather content from brochure-relevant pages
        for link in brochure_links:
            result += f"\n\n{link['type']}:\n"
            full_url = urljoin(url, link["url"])
            result += Website(full_url).get_contents()

        return result

    @staticmethod
    def get_brochure_user_prompt(company_name: str, url: str) -> str:
        """
        Generate a user prompt for the OpenAI model to create a brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        :return: A string containing the user prompt for brochure generation
        """
        user_prompt = f"You are looking at a company called: {company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += BrochureGenerator.get_all_details(url)
        return user_prompt[:20_000]  # Truncate if more than 20,000 characters

    @staticmethod
    def stream_brochure(company_name: str, url: str):
        """
        Generate and stream a company brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        """
        # Make a streaming API call to OpenAI for brochure generation
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": BrochureGenerator.SYSTEM_PROMPT},
                {"role": "user", "content": BrochureGenerator.get_brochure_user_prompt(company_name, url)}
            ],
            stream=True
        )

        # Display the generated brochure in real-time
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


API key looks good so far


In [8]:
# Main execution block
if __name__ == "__main__":
    # Generate a brochure
    BrochureGenerator.stream_brochure("Edward Donner", "https://edwarddonner.com/")

Found Brochure links: [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]


# Edward Donner Brochure

## About Us
Edward Donner is a visionary in the field of artificial intelligence, specializing in applying AI to solve real-world problems, particularly in talent acquisition and management. As the co-founder and CTO of Nebula.io, Edward is dedicated to helping people discover their potential and pursue fulfilling careers. His journey began with the founding of untapt, an AI startup acquired in 2021, which laid the groundwork for Nebula.io's innovative approach to recruitment.

## What We Do
At Nebula.io, we leverage groundbreaking, proprietary large language models (LLMs) verticalized for talent. Our patented matching model revolutionizes the way recruiters source, understand, engage, and manage talent. By eliminating the need for keywords, we provide faster and more accurate matches, helping individuals find roles where they can thrive and succeed.

## How We Do It
Our approach is rooted in the concept of Ikigai, a Japanese philosophy that emphasizes finding one's reason for being. We use Generative AI and machine learning to match individuals with roles that align with their passions, skills, and aspirations. This not only enhances individual fulfillment but also raises the level of human prosperity by ensuring that more people are engaged and inspired in their work.

## Where We Do It
Based in New York City, Nebula.io operates at the intersection of technology and human potential. Our innovative solutions are used by recruiters worldwide, helping them to connect with top talent and manage their recruitment processes more effectively.

## Our People
Edward Donner leads a team of passionate and skilled professionals who are committed to making a positive impact through AI. With expertise spanning software engineering, data science, technology leadership, and entrepreneurship, our team is dedicated to pushing the boundaries of what AI can achieve.

## Our Culture
At Nebula.io, we foster a culture of innovation, collaboration, and continuous learning. We believe in the power of AI to transform lives and are driven by a shared commitment to helping people discover their potential. Our team is united by a passion for solving real-world problems and a belief in the importance of creating meaningful connections.

## Connect with Us
We invite you to connect with Edward Donner and the Nebula.io team to learn more about our work and explore potential collaborations. Whether you're a prospective customer, investor, or recruit, we'd love to hear from you. Reach out to us at ed [at] edwarddonner [dot] com or follow us on LinkedIn, Twitter, and Facebook.

---

# エドワード・ドナー パンフレット

## 私たちについて
エドワード・ドナーは、人工知能（AI）の分野で特に人材採用と管理にAIを応用するビジョナリーです。Nebula.ioの共同創業者兼CTOとして、エドワードは人々が自分の可能性を見つけ、充実したキャリアを追求する手助けをすることを使命としています。彼の旅は、2021年に買収されたAIスタートアップuntaptの創業から始まり、Nebula.ioの革新的な採用アプローチの基盤を築きました。

## 私たちの仕事
Nebula.ioでは、人材向けに特化した画期的な独自の大規模言語モデル（LLM）を活用しています。私たちの特許取得済みのマッチングモデルは、採用担当者が人材を発掘し、理解し、関与し、管理する方法を革新します。キーワードを必要とせず、より速く正確なマッチングを提供し、個人が成功しやすい役割を見つける手助けをします。

## 私たちの方法
私たちのアプローチは、日本の哲学である「生き甲斐（Ikigai）」に根ざしています。生成AIと機械学習を使用して、個人の情熱、スキル、願望に合った役割を見つけます。これにより、個人の満足度が向上し、より多くの人々が仕事に情熱を持ち、インスピレーションを受けることで、人間の繁栄のレベルが向上します。

## 私たちの活動場所
ニューヨーク市を拠点とするNebula.ioは、テクノロジーと人間の可能性の交差点で活動しています。私たちの革新的なソリューションは、世界中の採用担当者によって使用され、トップ人材との接続と採用プロセスの管理をより効果的に支援しています。

## 私たちのチーム
エドワード・ドナーは、AIを通じてポジティブな影響を与えることに情熱を持ち、スキルの高い専門家チームを率いています。ソフトウェアエンジニアリング、データサイエンス、テクノロジーリーダーシップ、起業家精神にわたる専門知識を持つ私たちのチームは、AIが達成できることの限界を押し広げることに専念しています。

## 私たちの文化
Nebula.ioでは、イノベーション、コラボレーション、継続的な学習の文化を育んでいます。私たちは、AIが人生を変える力を持っていると信じており、人々が自分の可能性を見つける手助けをするという共通のコミットメントに駆り立てられています。私たちのチームは、現実世界の問題を解決することと、意味のあるつながりを作ることの重要性を信じて団結しています。

## お問い合わせ
エドワード・ドナーとNebula.ioチームとのつながりを持ち、私たちの仕事についてもっと学び、潜在的なコラボレーションを探ることをお勧めします。あなたが将来の顧客、投資家、または採用候補者であれば、ぜひお聞かせください。ed [at] edwarddonner [dot] comまでご連絡いただくか、LinkedIn、Twitter、Facebookでフォローしてください。

---

# 爱德华·唐纳公司手册

## 关于我们
爱德华·唐纳是人工智能领域的远见者，专注于将AI应用于解决现实世界的问题，特别是在人才招聘和管理方面。作为Nebula.io的联合创始人兼首席技术官，爱德华致力于帮助人们发现自己的潜力并追求充实的职业生涯。他的旅程始于2013年创立的AI初创公司untapt，该公司于2021年被收购，为Nebula.io的创新招聘方法奠定了基础。

## 我们的工作
在Nebula.io，我们利用专为人才量身定制的开创性专有大语言模型（LLM）。我们获得专利的匹配模型彻底改变了招聘人员寻找、理解、吸引和管理人才的方式。通过消除对关键词的需求，我们提供了更快、更准确的匹配，帮助个人找到能够茁壮成长并取得成功的角色。

## 我们的方法
我们的方法根植于“Ikigai”这一日本哲学，强调找到自己的存在意义。我们使用生成式AI和机器学习来匹配个人的激情、技能和愿望。这不仅增强了个人成就感，还通过确保更多人在工作中感到投入和受到启发，提高了人类的繁荣水平。

## 我们的工作地点
Nebula.io总部位于纽约市，处于技术与人类潜力的交汇点。我们的创新解决方案被全球的招聘人员使用，帮助他们与顶尖人才建立联系并更有效地管理招聘流程。

## 我们的团队
爱德华·唐纳领导着一支充满激情和技能的团队，致力于通过AI产生积极影响。我们的团队在软件工程、数据科学、技术领导和创业精神方面拥有广泛的专业知识，致力于突破AI的界限。

## 我们的文化
在Nebula.io，我们培养创新、协作和持续学习的文化。我们相信AI有改变生活的力量，并致力于帮助人们发现自己的潜力。我们的团队团结一致，致力于解决现实世界的问题，并相信建立有意义的联系的重要性。

## 联系我们
我们邀请您与爱德华·唐纳和Nebula.io团队联系，了解更多关于我们的工作并探索潜在的合作机会。无论您是潜在客户、投资者还是招聘对象，我们都希望听到您的声音。请通过ed [at] edwarddonner [dot] com与我们联系，或在LinkedIn、Twitter和Facebook上关注我们。

---

# Folleto de Edward Donner

## Sobre Nosotros
Edward Donner es un visionario en el campo de la inteligencia artificial, especializado en aplicar IA para resolver problemas del mundo real, particularmente en la adquisición y gestión de talento. Como cofundador y CTO de Nebula.io, Edward está dedicado a ayudar a las personas a descubrir su potencial y perseguir carreras satisfactorias. Su viaje comenzó con la fundación de untapt, una startup de IA adquirida en 2021, que sentó las bases para el enfoque innovador de Nebula.io en la contratación.

## Qué Hacemos
En Nebula.io, aprovechamos modelos de lenguaje grandes (LLM) propietarios y revolucionarios, verticalizados para el talento. Nuestro modelo de coincidencia patentado revoluciona la forma en que los reclutadores buscan, entienden, involucran y gestionan el talento. Al eliminar la necesidad de palabras clave, proporcionamos coincidencias más rápidas y precisas, ayudando a las personas a encontrar roles en los que puedan prosperar y tener éxito.

## Cómo Lo Hacemos
Nuestro enfoque está enraizado en el concepto de Ikigai, una filosofía japonesa que enfatiza encontrar la razón de ser de uno. Utilizamos IA generativa y aprendizaje automático para emparejar a las personas con roles que se alinean con sus pasiones, habilidades y aspiraciones. Esto no solo mejora la realización individual, sino que también eleva el nivel de prosperidad humana al asegurar que más personas estén comprometidas e inspiradas en su trabajo.

## Dónde Lo Hacemos
Con sede en la ciudad de Nueva York, Nebula.io opera en la intersección de la tecnología y el potencial humano. Nuestras soluciones innovadoras son utilizadas por reclutadores en todo el mundo, ayudándoles a conectarse con el mejor talento y gestionar sus procesos de contratación de manera más efectiva.

## Nuestra Gente
Edward Donner lidera un equipo de profesionales apasionados y capacitados que están comprometidos a generar un impacto positivo a través de la IA. Con experiencia en ingeniería de software, ciencia de datos, liderazgo tecnológico y emprendimiento, nuestro equipo está dedicado a expandir los límites de lo que la IA puede lograr.

## Nuestra Cultura
En Nebula.io, fomentamos una cultura de innovación, colaboración y aprendizaje continuo. Creemos en el poder de la IA para transformar vidas y estamos impulsados por un compromiso compartido de ayudar a las personas a descubrir su potencial. Nuestro equipo está unido por una pasión por resolver problemas del mundo real y una creencia en la importancia de crear conexiones significativas.

## Conéctate con Nosotros
Te invitamos a conectarte con Edward Donner y el equipo de Nebula.io para conocer más sobre nuestro trabajo y explorar posibles colaboraciones. Ya sea que seas un cliente potencial, inversor o recluta, nos encantaría saber de ti. Contáctanos en ed [at] edwarddonner [dot] com o síguenos en LinkedIn, Twitter y Facebook.